In [1]:
import pandas as pd
import math

In [2]:
cereal_df = pd.read_csv("globalterrorismdb_0718dist.csv", encoding = "ISO-8859-1")

/Users/austin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [72]:
post_year = 2017
month_dict = {}
gname_occ = {}
for i, row in cereal_df.iterrows():
    year = row['iyear']
    month = row['imonth']
    gname = row['gname']
    if year >= post_year and gname != 'Unknown':
        lat = row['latitude']
        if not math.isnan(lat):
            day = row['iday']
            if gname not in gname_occ:
                gname_occ[gname] = 0
            gname_occ[gname] += 1

            month_key = str(year) + '-' + str(month).zfill(2) + '-02'
            if month_key not in month_dict:
                month_dict[month_key] = {}
            if gname not in month_dict[month_key]:
                month_dict[month_key][gname] = 0
            month_dict[month_key][gname] += 1

In [73]:
_gname_occ = {k: v for k, v in sorted(gname_occ.items(), key=lambda item: item[1], reverse=True)}

In [74]:
i = 0
topK = 10
topK_groups = []
for k, v in _gname_occ.items():
    i += 1
    if ',' in k:
        print('need to handle , in group name')
    topK_groups.append(k)
    if i >= topK:
        break

In [75]:
topK_groups

['Islamic State of Iraq and the Levant (ISIL)',
 'Taliban',
 'Al-Shabaab',
 "New People's Army (NPA)",
 'Boko Haram',
 'Maoists',
 'Khorasan Chapter of the Islamic State',
 "Kurdistan Workers' Party (PKK)",
 'Houthi extremists (Ansar Allah)',
 'Sinai Province of the Islamic State']

In [76]:
with open('../month_gname.csv', 'w') as f:
    l = 'date'
    for gname in topK_groups:
        if ',' in gname:
            gname = '"' + gname + '"'

        l += ',' + gname
    f.write(l + '\n')
    
    for date in month_dict:
        l = date
        for gname in topK_groups:
            if gname in month_dict[date]:
                l += ',' + str(month_dict[date][gname])
            else:
                l += ',0'
        f.write(l + '\n')

In [69]:
date_coord = []
for i, row in cereal_df.iterrows():
    year = row['iyear']
    month = row['imonth']
    gname = row['gname']
    if year >= post_year and gname in topK_groups:
        day = row['iday']
        lat = row['latitude']
        lon = row['longitude']
        kill = row['nkill']
        attype = row['attacktype1_txt']
        if not math.isnan(lat):
            if math.isnan(kill):
                kill = 0
                
            date = str(year) + '-' + str(month).zfill(2) + '-' + str(day).zfill(2)
            l = date + ',' + gname + ',' + str(lon) + ',' + str(lat) + ',' + str(kill) + ',' + str(attype) +'\n'
            date_coord.append(l)

In [71]:
with open('../date_coord.csv', 'w') as f:
    l = 'date,gname,longitude,latitude,value,attype\n'
    f.write(l)
    for l in date_coord:
        f.write(l)

In [66]:
gname_attype = {}
attype_occ = {}
for i, row in cereal_df.iterrows():
    year = row['iyear']
    month = row['imonth']
    gname = row['gname']
    if year >= post_year and gname in topK_groups:
        day = row['iday']
        lat = row['latitude']
        attype = row['attacktype1_txt']
        if not math.isnan(lat):
            day = row['iday']
            if attype not in attype_occ:
                attype_occ[attype] = 0
            attype_occ[attype] += 1

            if gname not in gname_attype:
                gname_attype[gname] = {}
            if attype not in gname_attype[gname]:
                gname_attype[gname][attype] = 0
            gname_attype[gname][attype] += 1

In [67]:
with open('../gname_attype.csv', 'w') as f:
    l = 'gname,attype,value\n'
    f.write(l)
    for gname in topK_groups:
        if ',' in gname:
            gname = '"' + gname + '"'        
        for attype in sorted(set(attype_occ)):
            l = gname + ',' + attype
            if attype in gname_attype[gname]:
                l += ',' + str(gname_attype[gname][attype])
            else:
                l += ',0'
            f.write(l + '\n')

In [68]:
sorted(set(attype_occ))

['Armed Assault',
 'Assassination',
 'Bombing/Explosion',
 'Facility/Infrastructure Attack',
 'Hijacking',
 'Hostage Taking (Barricade Incident)',
 'Hostage Taking (Kidnapping)',
 'Unarmed Assault',
 'Unknown']